# Basis gegevensanalyse
## Download het bestand
We beginnen met de data te downloaden

In [ ]:
from pathlib import Path
import requests
OCCUPATION_CSV = 'occupation.csv'
occupation_path = Path(OCCUPATION_CSV)
if not occupation_path.exists():
    URL = 'https://raw.githubusercontent.com/justmarkham/DAT8/master/data/u.user'

    data = requests.get(URL)
    with open(OCCUPATION_CSV, mode='w') as f:
        f.write(data.text)
else:
    print('bestand bestaat al')

## Algemene structuur controleren
Ik lees de eerste regels met Python om een idee te krijgen van de structuur. Ik zou het ook kunnen openen in kladblok, natuurlijk. 

In [ ]:
OCCUPATION_CSV = 'occupation.csv'
with open(OCCUPATION_CSV) as f:
    for _ in range(5):
        print(f.readline(), end='')

## Inlezen in pandas
Nu ik de delimiter ken, kan ik het bestand in pandas lezen, de info() opvragen en de eerste regels tonen

In [ ]:
import pandas as pd
OCCUPATION_CSV = 'occupation.csv'
df = pd.read_csv(OCCUPATION_CSV, delimiter='|', index_col='user_id')
df.info()
print(df.head())

## Hoe groot is het dataframe?
Om het aantal rijen te weten, kunnen we shape[0] gebruiken (943 dus)

In [ ]:
df.shape[0]

## Juiste types definiëren
We zijn geen fan van objecten, dus we zullen het pandas StringDType gebruiken

In [ ]:
keys = df.columns[1:]
types = {key:pd.StringDtype() for key in keys}
df = pd.read_csv(OCCUPATION_CSV, delimiter='|', index_col='user_id', dtype=types)
df.info(memory_usage='deep')

## Wat zijn de waarden
Om een idee te krijgen van de waarden voor elke kolom, kunnen we de describe()-functie gebruiken. Standaard toont die alleen informatie voor de getalkolommen. Maar met *include='all'* krijgen we alle kolommen te zien.

Bij *age* zien we dat een kwart van de mensen jonger is dan 25 jaar.

In [ ]:
df.describe(include='all')

# Werken met de unieke waarden
We kunnen de unieke waarden ook apart opvragen met nunique(). De functie value_counts() vertelt ons hoe dikwijls elke beroep voorkomt. Wanneer we sorteren, kunnen we een antwoord geven op de vraag: welk beroep komt het minst voor. 

Met tail(3) kunnen we de drie beroepen opvragen die het minst voorkomen.

In [ ]:
print('unieke beroepen:', df.occupation.nunique())
print('Welke beroepen komen het minst voor?\n', df.occupation.value_counts().sort_values(ascending=True))
print('Wat zijn de drie beroepen die het minst voorkomen?\n', df.occupation.value_counts().tail(3).sort_values())

## Hoe zit het met de percentages mannen en vrouwen
We kunnen dat 'manueel' berekenen

In [ ]:
df['gender'].value_counts() / df.shape[0] * 100

Maar .value_counts() kan dat ook zelf

In [ ]:
df['gender'].value_counts(normalize=True) * 100

## Leeftijdsverdeling
We kunnen de statistische eigenschappen van de getalkolom ook opvragen met functies:

In [ ]:
print('Gemiddelde is', df.age.mean())
print('Standaardafwijking is', df.age.std())
print('Laagste waarde is', df.age.min())
print('Grensleeftijd eerste kwartiel', df.age.quantile(0.25))
print('Mediaan', df.age.median())
print(' of Grensleeftijd tweede kwartiel', df.age.quantile(0.5))
print('Grensleeftijd derde kwartiel', df.age.quantile(0.75))
print('Hoogste waarde', df.age.max())

## Hoeveel unieke waarden voor gender

In [ ]:
df.gender.nunique()

## De leeftijdsverdeling wordt duidelijker met een plot
Een histogram is aangewezen wanneer we een aantal getalwaarden in 'bins' willen tonen.

In [ ]:
df.age.plot(kind='hist')

## En met pyplot
Wanneer we de kwartielen als grenswaarden nemen voor de bins, zien we dat de leeftijden gelijk verdeeld zijn. 

In [ ]:
import matplotlib.pyplot as plt
plt.hist(df.age,[7, 25, 31, 43, 73])
plt.show()

## Welke beroepen beoefent iemand die 7 jaar oud is?
Ten eerste zullen we eens zien hoeveel minderjarigen er zijn. (laat ons 18 definiëren als minderjarig). Vervolgens kunnen we eens zien welke beroepen ze uitoefenen

In [ ]:
print(df[df['age'] < 18])

## Student is geen beroep
Student is natuurlijk geen echt beroep. We zitten ook met het beroep *none* (tekst). Ik ga die data verwijderen.

In [ ]:
df_beroepen = df[(df['occupation'] != 'student') & (df['occupation'] != 'none')]
df_beroepen.describe(include='all')

## Categorietype
Wanneer we een kolom hebben met een beperkt aantal strings, kunnen we het pandas categorietype gebruiken voor die kolom. Laat ons dat eens proberen met 'gender'. We zien dat de grootte van het dataframe geminderd is van 163 KB naar 118 KB.

Let op met categorical data wanneer we bepaalde manipulaties willen doen, zoals waarden hernoemen, dataframes samenvoegen, groeperen, ... (zie [Using pandas categories properly is tricky, here’s why…](https://medium.com/data-science/staying-sane-while-adopting-pandas-categorical-datatypes-78dbd19dcd8a))

In [ ]:
df = df.astype({'gender':'category'})
df.info(memory_usage='deep')

# Categorie voor occupation
Ook *occupation* bestaat uit een beperkt aantal waarden. Omdat de strings in *occupation* langer zijn dan M en F, zien we nu een grotere geheugenwinst (nog 86 KB)

In [ ]:
df = df.astype({'occupation':'category'})
df.info(memory_usage='deep')